Elastic Modulus and Poissons Ratio Fitting & Viewer

In [ ]:
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt; import numpy as np
from pvc_tensile_experiment.Functions import *

def LinearFit(x, a, b):
    return a*x + b

# create a dataframe to save the data 
df = pd.DataFrame(columns = ["Plasticizer", "Elastic Modulus (Pa)", "Poissons Ratio", "Bulk Modulus"])
i = 0

# plot parameters
markerSize = 0.5
titleSize = 15
axisSize = 11
legendSize = 11

# the desired plasticizer ratio 
for plastiRatio in ['P2', 'P4', 'P6', 'P8']:

    # compile all the data  
    _, axStrain, _, transStrain, stress = InstronDataCompile('Elastic Data', plastiRatio)

    # define the elastic region
    elasRegion = np.where(axStrain < 0.3) 

    # curve fit the elastic region. 
    xfit = np.linspace(axStrain[elasRegion][0], axStrain[elasRegion][-1])
    poptE, pcovE = curve_fit(LinearFit, axStrain[elasRegion], stress[elasRegion]*1e6)
    poptP, pcovP = curve_fit(LinearFit, axStrain[elasRegion], transStrain[elasRegion])

    # plot engineering stress vs strain
    plt.figure(0)
    plt.scatter(axStrain[elasRegion], stress[elasRegion]*1e6, s = markerSize, label = f'{plastiRatio} Gel')
    # plt.plot(xfit, LinearFit(xfit, poptE[0], poptE[1]), 'black')
    plt.xlabel('Axial Strain', fontsize = axisSize)
    plt.ylabel('Stress (Pa)', fontsize = axisSize)
    plt.title(f'PVC Gel Elastic Modulus (Pa)', fontsize = titleSize)
    plt.legend(fontsize = legendSize)

    # plot poissons ratio
    plt.figure(1)
    plt.scatter(axStrain[elasRegion], transStrain[elasRegion], s = markerSize, label = f'{plastiRatio} Gel')
    # plt.plot(xfit, LinearFit(xfit, poptP[0], poptP[1]), 'black', )
    plt.xlabel('Axial Strain', fontsize = axisSize)
    plt.ylabel('Transverse Strain', fontsize = axisSize)
    plt.title(f'PVC Gel Poissons Ratio', fontsize = titleSize)
    plt.legend(fontsize = legendSize)

    df.loc[i] = [plastiRatio, poptE[0], poptP[0], poptE[0]/2/(1 - 2*poptP[0])]
    i = i + 1
plt.show()

# df.to_csv(f"Data/PVC Gel Material Parameters.csv", sep = ',', header = True, index = False)

Elastic Modulus Data Excel Combiner & Exporter

In [ ]:
from pvc_tensile_experiment.Functions import *; import numpy as np


# the desired plasticizer ratio 
for plastiRatio in ['P2', 'P4', 'P6', 'P8']:

    # compile all the data  
    _, axStrain, _, transStrain, stress = InstronDataCompile('Elastic Data', plastiRatio)

    # find the elastic region
    elasRegion = np.where(axStrain < 0.3) 

    # save the nominal stress and stretch data 
    df = pd.DataFrame(columns = ["Transverse Strain", "Eng. Axial Strain", "Eng. Stress (Pa)"])
    df["Transverse Strain"] = transStrain[elasRegion]
    df["Eng. Axial Strain"] = axStrain[elasRegion]
    df["Eng. Stress (Pa)"] = stress[elasRegion]    
    
    # df.to_csv(f"Data/{plastiRatio} Uniaxial Tensile Data.csv", sep = ',', header = True, index = False)


Hyperelastic Data and Incompressibilty Assumption Viewer

In [ ]:
from pvc_tensile_experiment.Functions import InstronDataCompile
import matplotlib.pyplot as plt; import numpy as np

# Compute R-squared
def RSquared(yTrue, yPredict):
    ssResidual = np.sum((yTrue - yPredict)**2)
    ssTotal = np.sum((yTrue - np.mean(yTrue))**2)
    return 1 - (ssResidual/ssTotal)

# preallocate the suplots
fig, axs1 = plt.subplots(1,1, layout = "constrained")
fig, axs2 = plt.subplots(1,1, layout = "constrained")
fig, axs3 = plt.subplots(1,1, layout = "constrained")

# plot parameters
markerSize = 0.5
titleSize = 15
axisSize = 11
legendSize = 11

# plot the ideal incompressability condition
stretch = np.arange(1.0, 4.5, 0.01)
axs1.plot(stretch, np.sqrt(1/stretch), label = 'Theoretical Fit', c = 'k')
axs1.legend(fontsize = 11)

# the desired plasticizer ratio 
for plastiRatio in ['P2', 'P4', 'P6', 'P8']:

    # read the processed data files 
    _, axStrain, _, transStrain, stress = InstronDataCompile(f'Hyperelastic Data/', plastiRatio)

    # find true strain and stress from engineering strain
    stretch = 1 + axStrain
    transverseStretch = 1 + transStrain
    stress = stress*1e6
    stressTrue = stress*stretch

    # print the coeff of determination per dataset
    coeffDet = RSquared( np.sqrt(1/stretch[np.isnan(stretch) == False]), -transverseStretch[np.isnan(stretch) == False] + 2)

    # plot inverse stretch ratio showing isotropic and incompressable
    axs1.scatter(stretch, -transverseStretch + 2, label = f'{plastiRatio}, $R^2$ = {np.round(coeffDet, 2)}', s = markerSize)
    axs1.set_xlabel('Axial Stretch', fontsize = axisSize)
    axs1.set_ylabel('Transverse Stretch', fontsize = axisSize)
    axs1.set_title("Incompressability Assumption", fontsize = titleSize)
    axs1.legend(fontsize = legendSize)

    # plot engineering stress-strain 
    axs2.scatter(axStrain*1e2, stress, s = markerSize, label = f'{plastiRatio}')   
    axs2.set_xlabel('Engineering Strain (%)', fontsize = axisSize)
    axs2.set_ylabel('Engineering Stress (Pa)', fontsize = axisSize)
    axs2.set_title('PVC Gel Engineering Stress', fontsize = titleSize)
    axs2.legend(fontsize = legendSize)
    
    # plot true stress-strain
    axs3.scatter(stretch, stressTrue, s = markerSize, label = f'{plastiRatio}')
    axs3.set_xlabel('Stretch', fontsize = axisSize)
    axs3.set_ylabel('True Stress (Pa)', fontsize = axisSize)
    axs3.set_title('PVC Gel True Stress', fontsize = titleSize)
    axs3.legend(fontsize = legendSize)
plt.show()

Hyperealastic Fitting

In [ ]:
"""True Stress- Strain Hyperelastic Model Fitting"""
import numpy as np; import scipy.optimize as opt
import matplotlib.pyplot as plt
from pvc_tensile_experiment.Functions import *

# define hyperelastic models
def NeoHookean(params, stretch):
    mu = params[0]
    return mu*(stretch**2 - 1/stretch)

def MooneyRivlin(params, stretch):
    C10, C01 = params
    return 2*(stretch**2 - 1/stretch)*(C10 + C01/stretch)

def Yeoh(params, stretch):
    C10, C20, C30 = params
    I1 = stretch**2 + 2/stretch
    return 2*(stretch**2 - 1/stretch)*(C10 + 2*C20*(I1 - 3) + 3*C30*(I1 - 3)**2)

def InvLangevin(x):
    if np.abs(x) < 0.84136:
        return 1.31446*np.tan(1.58986*x) + 0.91209*x
    elif np.abs(x) >= 0.84136:
        return 1/(np.abs(x)/x*-x)

def EightChain(params, stretch):
    mu, stretchLock = params
    chainStretch = np.sqrt((stretch**2 + 2/stretch)/3)    
    return mu/chainStretch*(stretch**2 - 1/stretch)*InvLangevin(chainStretch/stretchLock)/InvLangevin(1/stretchLock)

def Ogden(params, stretch):
    mu, alpha, mu1, alpha1, mu2, alpha2 = params
    return 2*mu/alpha*(stretch**(alpha) - stretch**(-alpha/2)) + 2*mu1/alpha1*(stretch**(alpha1) - stretch**(-alpha1/2)) + 2*mu2/alpha2*(stretch**(alpha2) - stretch**(-alpha2/2))

def Gent(params, stretch):
    mu, Jm = params
    return mu*(stretch**2 - 1/stretch)*(Jm/(Jm - (stretch**2 + 2/stretch - 3)))

def ExtendedTube(params, stretch):
    Gc, Ge, beta, delta = params
    I1 = stretch**2 + 2/stretch
    return Gc*(stretch**2 - 1/stretch)*((1 + (1 + I1**2 - 4*I1)*delta**2 + (5*I1 - I1**2 - 6)*delta**4)/(1 - (I1 - 3)*delta**2)**2) - 2*Ge/beta*(stretch**(-beta) - stretch**(beta/2))


# define the model fitting function
def FitModel(model, stretch, trueStress, boundaries):
    # define the residual function and the NRMS function
    def Residuals(params):
        return model(params, stretch) - trueStress

    def NRMS(params):
        return np.sqrt(np.mean(Residuals(params)**2))/np.sqrt(np.mean(trueStress**2))

    # run the genetic algorithm first and use the results for the levenberg-marquardt solution
    initialGuess = opt.differential_evolution(NRMS, bounds = boundaries, popsize = 1000).x
    optParams, _ = opt.least_squares(Residuals, initialGuess, method = 'lm', max_nfev = 1000).x, None
    print(f'    GA params: {initialGuess}')
    print(f'    Lm params: {optParams}')

    # show the solution of both solvers
    # plt.plot(stretch, model(initialGuess, stretch), label = f'GA Solution, $R^2$: {RSquared(trueStress, model(initialGuess, stretch)):.5f}')
    # plt.plot(stretch, model(optParams, stretch), label = f'Lm Solution, $R^2$: {RSquared(trueStress, model(optParams, stretch)):.5f}')
    # plt.scatter(stretch, trueStress, s = markerSize, label = "Data", color = "black")
    # plt.xlabel("Stretch Ratio", fontsize = axisSize)
    # plt.ylabel("True Stress", fontsize = axisSize)
    # plt.legend(fontsize = legendSize)
    # plt.show()
    return optParams

# compute R-squared
def RSquared(yTrue, yPredict):
    ssResidual = np.sum((yTrue - yPredict)**2)
    ssTotal = np.sum((yTrue - np.mean(yTrue))**2)
    return 1 - (ssResidual/ssTotal)


# define the plasticizer ration we want
plastiRatio = ['P2', 'P4', 'P6', 'P8']

for i in plastiRatio:
    # compile all the data  
    _, axStrain, _, transStrain, stress = InstronDataCompile('Hyperelastic Data', i)
    indices = np.where(axStrain < 2) 
    indices = range(0, indices[0][-1], 25)

    # convert to true stress-strain
    stretch = 1 + axStrain[indices]
    trueStress = stress[indices]*stretch*1e6

    # plot parameters
    markerSize = 0.5
    titleSize = 15
    axisSize = 11
    legendSize = 11

    # models and initial guesses
    models = {"Neo-Hookean": (NeoHookean, [(0.1, 1e7)]),
                "Mooney-Rivlin": (MooneyRivlin, ([1e3, 1e6], [-1e6, 1e4])),
                "Yeoh": (Yeoh,  ([1e2, 1e6], [-1e5, -1e1], [1e1, 1e3])),
                "Gent": (Gent, ([1e1 , 1e6], [1, 200])),
                # # "Arruda-Boyce": (EightChain, [1.0, 10]),
                # "Ogden": (Ogden, ([1e3, 1e7], [-5, 5], [1e2, 1e6], [-1, 3], [1e1, 1e6], [1, 5])),
                "Extended Tube": (ExtendedTube, ([1, 10.0e6], [100, 5e6], [0.1, 10], [1e-6, 0.1]))
                }

    # Fit models
    fitResults = {}

    for name, (model, boundaries) in models.items():
        print(name)
        params = FitModel(model, stretch, trueStress, boundaries)
        r2 = RSquared(trueStress, model(params, stretch))
        fitResults[name] = (params, r2)


    # Plot results
    plt.figure(figsize = (8, 6))
    plt.scatter(stretch, trueStress, s = markerSize, label = "Data", color = "black")

    for name, (params, r2) in fitResults.items():
        plt.plot(stretch, models[name][0](params, stretch), label = f'{name}, $R^2$: {np.round(r2, 3)}')

    plt.xlabel("Stretch Ratio", fontsize = axisSize)
    plt.ylabel("True Stress", fontsize = axisSize)
    plt.legend(fontsize = legendSize)
    plt.title(f'{i} PVC Gel Hyperelastic Model Fit', fontsize = titleSize)
    plt.show()

MISC. & Old Functions

In [ ]:
"""Nominal Engineering-Stretch Hyperelastic Model Fitting"""
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
from pvc_tensile_experiment.Functions import *


# define the plasticizer ration we want
plastiRatio = 'P2'

# compile all the data  
_, axStrain, _, _, stress = InstronDataCompile('Hyperelastic Data', plastiRatio)

# convert from engineering measurements to true measurements
stretch = 1 + axStrain
stress = stress*1e6

# Define hyperelastic models
def NeoHookean(params, stretch):
    mu = params[0]
    return mu*(stretch - 1/stretch**2)

def MooneyRivlin(params, stretch):
    C10, C01 = params
    return 2*(stretch - 1/stretch**2)*(C10 + C01/stretch)

def Yeoh(params, stretch):
    C10, C20, C30 = params
    I1 = stretch**2 + 2/stretch
    return 2*(stretch - 1/stretch**2)*(C10 + 2*C20*(I1 - 3) + 3*C30*(I1 - 3)**2)

def InvLangevin(x):
    if np.abs(x) < 0.84136:
        return 1.31446*np.tan(1.58986*x) + 0.91209*x
    elif np.abs(x) >= 0.84136:
        return 1/(np.abs(x)/x*-x)

def EightChain(params, stretch):
    mu, stretchLock = params
    chainStretch = np.sqrt((stretch**2 + 2/stretch)/3)    
    return mu/chainStretch*(stretch**2 - 1/stretch)*InvLangevin(chainStretch/stretchLock)/InvLangevin(1/stretchLock)

def Ogden(params, stretch):
    mu, alpha, mu1, alpha1, mu2, alpha2 = params
    return 2*mu/alpha*(stretch**(alpha - 1) - stretch**(-alpha/2 - 1)) + 2*mu1/alpha1*(stretch**(alpha1 - 1) - stretch**(-alpha1/2 - 1)) + 2*mu2/alpha2*(stretch**(alpha2 - 1) - stretch**(-alpha2/2 - 1))

def Gent(params, stretch):
    mu, Jm = params
    return mu*(stretch - 1/stretch**2)*(Jm/(Jm - (stretch**2 + 2/stretch - 3)))

def ExtendedTube(params, stretch):
    Gc, Ge, beta, delta = params
    I1 = stretch**2 + 2/stretch
    return Gc*(stretch - 1/stretch**2)*((1 + (1 + I1**2 - 4*I1)*delta**2 + (5*I1 - I1**2 - 6)*delta**4)/(1 - (I1 - 3)*delta**2)**2) - 2*Ge/beta*(stretch**(-beta) - stretch**(beta/2))


# Define fitting function
def FitModel(model, stretch, stress, paramGuess):
    def residuals(params):
        return model(params, stretch) - stress
    optParams, _ = opt.least_squares(residuals, paramGuess, method = 'lm', max_nfev = 1000).x, None
    return optParams

# Compute R-squared
def RSquared(yTrue, yPredict):
    ssResidual = np.sum((yTrue - yPredict)**2)
    ssTotal = np.sum((yTrue - np.mean(yTrue))**2)
    return 1 - (ssResidual/ssTotal)


# Models and initial guesses
models = {
    "Neo-Hookean": (NeoHookean, [1.0e6]),
    "Mooney-Rivlin": (MooneyRivlin, [1.0e6, 0.1e6]),
    "Yeoh": (Yeoh, [1.0e6, -0.1e4, 0.01e2]),
    "Gent": (Gent, [1.0e1, 100e6]),
    # "Arruda-Boyce": (EightChain, [1.0, 10]),
    "Ogden": (Ogden, [10.0e4, 1.0, -7.0e4, 2.0,  5.0e4, 2.0]),
    "ExtendedTube": (ExtendedTube, [1.0e6, 1.0e6, 0.7, 0.1])
}

# Fit models
fitResults = {}
for name, (model, guess) in models.items():
    params = FitModel(model, stretch, stress, guess)
    predictedStress = model(params, stretch)
    r2 = RSquared(stress, predictedStress)
    fitResults[name] = (params, r2)

# Print results
for name, (params, r2) in fitResults.items():
    print(f"{name} Model: Parameters = {params}, R^2 = {r2:.4f}")

# Plot results
plt.figure(figsize = (8, 6))
plt.scatter(stretch, stress, s = 2, label = "Data", color = "black")

for name, (params, _) in fitResults.items():
    plt.plot(stretch, models[name][0](params, stretch), label = name, linewidth = 2, linestyle = 'dotted')

plt.xlabel("Stretch Ratio")
plt.ylabel("Nominal Stress (MPa)")
plt.legend()
plt.title("Hyperelastic Model Fits")
plt.show()
